In [1]:
# Import stuff
import pystan
import matplotlib.pyplot as plt
import numpy as np
import time
import scipy.interpolate as interpolate
import netCDF4
from models.stan_dlm_models import *
import tqdm
%matplotlib inline

In [2]:
# Compile the DLM model: this translates the stan model in stan_dlm_models.py into C++ and compiles it
# Note: this step may take a few minutes
model_kalman_ar1 = pystan.StanModel(model_code=code_kalman_ar1)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_ce0cf070d48949045c8df5a0c7d92012 NOW.


In [3]:
# Import the data 
# You can do this however you like but the end result must be the following variables loaded in:
# N (int) = number of time-steps in the time-series
# d float[N] = the data time-series (IMPORTANT: NaNs are NOT allowed, for missing values please set those data to zero and set
# the corresponding error for those data points to something very large, like 10^20)
# s float[N] = std-deviation error-bars on each data point (set error bars for missing values to a large number; see above)

# Import data from a netCDF
data = netCDF4.Dataset('data/BASIC_V1_2017_lotus_seascyc_gcsw2017_fac2.nc')

# Extract time, pressure and latitude variables from the netCDF
T = data['time'][:]
P = data['pressure'][:]
L = data['latitude'][:]

# How many time steps are there? (ie how long is the time-series)
N = len(T)

In [4]:
# Import the regressors

# Again you can do this however you want, but the result must be the following variables loaded into python
# nreg (int) = number of regressors
# regressors float[N, nreg] = 2d array with each column representing one regressor over the times corresponding to
# the data to be analysed

# NOTE: We also mean subtract and rescale all regressors in the same way as for the data so they have dymanic range 
# of 1 

# IMPORTANT: Missing values/NaNs are NOT supported, please interpolate missing values or similar so they are all 
# real valued

# ENSO
y = np.loadtxt('regressors/ENSO_MEI_ENSO_1950_201802.dat')
t = np.loadtxt('regressors/ENSOTime_MEI_ENSO_1950_201802.dat')
y = y - np.mean(y)
y = y/(max(y) - min(y))
Y = interpolate.InterpolatedUnivariateSpline(t, y)
enso = Y(T) # This extacts the regressor at the times you have in your dataset, interpolated in case the time grid is different to the actual data

# Solar
y = np.loadtxt('regressors/Flux_F30_monthly_195111_201803_absolute.dat')
t = np.loadtxt('regressors/Time_F30_monthly_195111_201803_absolute.dat')
y = y - np.mean(y)
y = y/(max(y) - min(y))
Y = interpolate.InterpolatedUnivariateSpline(t, y)
solar = Y(T)

# QBO30
y = np.loadtxt('regressors/multi_qbo30_1953_2018.dat')
t = np.loadtxt('regressors/multi_qbotime_1953_2018.dat')
y = y - np.mean(y)
y = y/(max(y) - min(y))
Y = interpolate.InterpolatedUnivariateSpline(t, y)
qbo30 = Y(T) 

# QBO50
y = np.loadtxt('regressors/multi_qbo50_1953_2018.dat')
t = np.loadtxt('regressors/multi_qbotime_1953_2018.dat')
y = y - np.mean(y)
y = y/(max(y) - min(y))
Y = interpolate.InterpolatedUnivariateSpline(t, y)
qbo50 = Y(T) # This extacts the regressor at the times you have in your dataset, interpolated in case the time grid is different

# SAOD
saod_data = netCDF4.Dataset('regressors/sad_1979_2017_10deg_60S_60N_missNans.nc')
# ***RESCALING WILL BE DONE ON THE FLY AS IT IS LATITUDE DEPENDENT***

# How many regressors?
nreg = 5

In [11]:
# How many MCMC chains/samples to do
n_chains = 1
warmup = 1000
iterations = 3000
n_samples = n_chains*(iterations-warmup)

# Where to save the results
trend = np.zeros((n_samples, len(P), len(L), len(T)))
slope = np.zeros((n_samples, len(P), len(L), len(T)))
seasonal = np.zeros((n_samples, len(P), len(L), len(T)))
residuals = np.zeros((n_samples, len(P), len(L), len(T)))
regressor_coefficients = np.zeros((n_samples, len(P), len(L), nreg))
sigma_trend = np.zeros((n_samples, len(P), len(L)))
sigma_seas = np.zeros((n_samples, len(P), len(L)))
sigma_AR = np.zeros((n_samples, len(P), len(L)))
rho_AR = np.zeros((n_samples, len(P), len(L)))

# Set up the progress bar
pbar = tqdm.tqdm_notebook(total = len(P)*len(L), desc = "press/lats")

# Loop over pressures and latitudes
for pressure in range(len(P)):
    for latitude in range(len(L)):

        # Set the data and initialization of parameters that are fed into the DLM

        # Pick out a pressure and latitude panel: this is the "data" time-series from the netCDF
        d = data['o3'][:, pressure, latitude]

        # Extract the error-bars on the time-series from the netCDF
        s = data['o3_sigma'][:, pressure, latitude]
        
        # Check if the data are there
        if np.mean(d) != 0:

            # IMPORTANT: The DLM is set-up to run on mean subtracted data re-scaled to have a dynamic range of unity
            # NB: We must re-scale the data here and re-scale back to the original units after the DLM run, look out for this later

            # Mean subtract and re-scale the data so it has dynamic range of unity, 
            # Apply same scaling transform to the std-dev error bars
            scale = max(d) - min(d)
            mean = np.mean(d)
            d = d - mean
            d = d/scale
            s = s/scale

            # Set the SAOD regressor depending on the latitude
            y = saod_data['saod_strat_column'][:, latitude]
            t = saod_data['time'][:]
            y = y - np.mean(y)
            y = y/(max(y) - min(y))
            Y = interpolate.InterpolatedUnivariateSpline(t, y)
            saod = Y(T)

            # Regressors
            regressors = np.column_stack([enso, solar, qbo30, qbo50, saod]) # Stack of all the regressors together in a 2d array

            # Data: this is a dictionary of all of the data/inputs that the DLM model needs (descriptions below)
            input_data = {
                                'd':d, # float[N] data vector
                                's':s, # float[N] std-dev error bars
                                'N':N, # (int) number of time-steps in the time-series
                                'nreg':nreg, # (int) number of regressors
                                'regressors':regressors, # float[N, nreg] the regressors
                                'S':10., # prior variance on the regression coefficients (priors are zero mean Gaussian with variance S)
                                'sigma_trend_prior':1e-4, # std-dev of the half-Gaussian prior on sigma_trend that controls how wiggly the trend can be
                                'sigma_seas_prior':0.01, # std-dev of the half-Gaussian prior on sigma_seas that controls how dynamic the seaonal cycle can be
                                'sigma_AR_prior':0.5 # std-dev of the half_Gaussian prior on the AR1 process std-dev 
                            }

            # Initialization: Initial guess values for the hyper-parameters
            initial_state = {
                             'sigma_trend':0.0001,
                             'sigma_seas':0.001,
                             'sigma_AR':0.01,
                             'rhoAR':0.1,
                            }

            # Run the model
            fit = model_kalman_ar1.sampling(data=input_data, iter=iterations, warmup=warmup, chains=n_chains, init = [initial_state for i in range(n_chains)], verbose=True, pars=('sigma_trend', 'sigma_seas', 'sigma_AR', 'rhoAR', 'trend', 'slope', 'beta', 'seasonal', 'residuals'))

            # Extract and re-scale the key bits from the DLM run

            # Trend
            trend[:, pressure, latitude, :] = fit.extract()['trend'][:,:]*scale + mean

            # Gradient of the DLM trend
            slope[:, pressure, latitude, :] = fit.extract()['slope'][:,:]*scale

            # Seasonal cycle
            seasonal[:, pressure, latitude, :] = fit.extract()['seasonal'][:,:]*scale

            # Residuals
            residuals[:, pressure, latitude, :] = fit.extract()['residuals'][:,:]*scale

            # Regressor coefficients (also need re-scaling back to data units)
            regressor_coefficients[:, pressure, latitude, :] = fit.extract()['beta'][:,:]*scale

            # DLM hyper parameters
            sigma_trend[:, pressure, latitude] = fit.extract()['sigma_trend']
            sigma_seas[:, pressure, latitude] = fit.extract()['sigma_seas']
            sigma_AR[:, pressure, latitude] = fit.extract()['sigma_AR']*scale # Also needs re-scaling back to data units
            rho_AR[:, pressure, latitude] = fit.extract()['sigma_AR']

        # Update the progress bar
        pbar.update(1)

HBox(children=(IntProgress(value=0, description='press/lats', max=324), HTML(value='')))

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


In [13]:
# Save all the results
np.save(open('results/trend.npy', 'wb'), trend)
np.save(open('results/slope.npy', 'wb'), slope)
np.save(open('results/seasonal.npy', 'wb'), seasonal)
np.save(open('results/residuals.npy', 'wb'), residuals)
np.save(open('results/regressor_coefficients.npy', 'wb'), regressor_coefficients)
np.save(open('results/sigma_trend.npy', 'wb'), sigma_trend)
np.save(open('results/sigma_seas.npy', 'wb'), sigma_seas)
np.save(open('results/sigma_AR.npy', 'wb'), sigma_AR)
np.save(open('results/rho_AR.npy', 'wb'), rho_AR)